In [ ]:
import asyncio
from polymarket.src.stream.binance import binance_price_ws
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from polymarket.src.db.ingest_stream import insert_into_db, parse_message

conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;')
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()


async def main(conf, queue, symbols, table_name):
    executor = ProcessPoolExecutor(max_workers=10)
    db_task = asyncio.create_task(insert_into_db(conf, table_name, queue, parse_message, executor))
    await binance_price_ws(queue=queue, symbols=symbols)
    await queue.put(None)
    await db_task

await main(conf, queue, symbols, table_name)

In [ ]:
import asyncio
from polymarket.src.stream.binance import binance_price_ws
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from polymarket.src.db.ingest_stream import insert_into_db, parse_message
conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;')
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()

async def main():

    executor = ThreadPoolExecutor(max_workers=10)
    producers = [asyncio.create_task(binance_price_ws(queue=queue, symbols=symbols)),]
    workers = [asyncio.create_task(insert_into_db(conf, table_name, queue, parse_message, executor))]

    await asyncio.gather(*producers, *workers)

await main()

Starting DB insert task...
Connected to Binance Futures Ticker WebSocket!
('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768670291068, 's': 'BTCUSDT', 'p': '508.00', 'P': '0.536', 'w': '95253.02', 'c': '95318.70', 'Q': '0.002', 'o': '94810.70', 'h': '95621.90', 'l': '94502.80', 'v': '52379.266', 'q': '4989283501.88', 'O': 1768583880000, 'C': 1768670291066, 'F': 7110279825, 'L': 7111446916, 'n': 1161822})
{'symbol': 'BTCUSDT', 'open': 94810.7, 'volume': 52379.266, 'timestamp_binance': 1768670291066}
('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768670292389, 's': 'BTCUSDT', 'p': '508.10', 'P': '0.536', 'w': '95253.02', 'c': '95318.80', 'Q': '0.002', 'o': '94810.70', 'h': '95621.90', 'l': '94502.80', 'v': '52379.319', 'q': '4989288553.78', 'O': 1768583880000, 'C': 1768670292387, 'F': 7110279825, 'L': 7111446922, 'n': 1161828})
{'symbol': 'BTCUSDT', 'open': 94810.7, 'volume': 52379.319, 'timestamp_binance': 1768670292387}
('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768670294410, 's': 'BTCUS